In [1]:
#import dependencies
import pandas as pd
import tensorflow as tf
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [3]:
#read in merged and cleaned data
df = pd.read_csv('../../data/merged_cleaned_data/cases_merged_data.csv')
df

,Unnamed: 0,FIPS,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_PCI,E_NOHSDP,...,Income Per Capita,Neuroticism,Openness,Religiosity,Risk Taking,Selflessness,Tolerance,Work Ethic,dem_pct,first_year_cases
0,0,1001,594.443459,55200,23315,21115,8422,1065,29372,4204,...,26168.0,77.925476,78.222354,91.106719,53.333333,82.142857,70.000000,60.380952,27.018365,6589
1,1,1009,644.830460,57645,24222,20600,8220,909,22656,7861,...,21033.0,78.764620,78.193105,92.045455,57.603815,79.307632,64.953288,76.000000,9.569378,6444
2,2,1013,776.838201,20025,10026,6708,4640,567,20430,2141,...,19011.0,78.563680,76.109761,76.623924,69.058104,79.956648,67.920284,72.773953,41.789629,2097
3,3,1015,605.867251,115098,53682,45033,20819,4628,24706,12620,...,22231.0,79.439032,79.955121,77.918741,54.063568,76.745724,67.456150,68.292794,29.845243,14224
4,4,1017,596.560643,33826,16981,13516,5531,773,22827,4383,...,21532.0,76.995358,78.156771,75.891100,67.343775,79.128558,66.397785,69.554441,41.644857,3488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3053,3053,48131,1793.476183,11355,5592,3511,2751,482,17864,2386,...,19853.0,79.125428,78.895880,76.629575,60.576045,73.670302,64.571017,68.007770,50.959826,1214
3054,3054,48505,998.411980,14369,6388,4405,5609,621,17228,3226,...,16007.0,79.355639,79.572483,74.378252,77.443239,76.386871,74.001471,73.609838,47.134744,1760
3055,3055,48507,1297.406535,12131,4344,3509,4150,421,13350,2719,...,13393.0,78.392216,76.024682,75.848196,76.967659,77.303576,70.010162,71.121990,65.403060,1844
3056,3056,48127,1328.884075,10663,4408,3309,3148,694,18121,2137,...,19528.0,79.457964,79.141556,66.677924,73.140121,71.659882,68.065499,66.378831,61.756683,1707


In [4]:
#set index to FIPS
df = df.set_index(df['FIPS'])
df= df.drop(columns = ['FIPS','Unnamed: 0'])

In [5]:
#create variable for case% for each counties population
df['case_pct'] = df['first_year_cases']/df['E_TOTPOP']*100
df['case_pct'].head()

FIPS
1001    11.936594
1009    11.178767
1013    10.471910
1015    12.358164
1017    10.311595
Name: case_pct, dtype: float64

In [6]:
df['case_pct'].describe()

count    3058.000000
mean        9.426600
std         3.045809
min         0.000000
25%         7.713422
50%         9.466675
75%        11.176131
max        38.010657
Name: case_pct, dtype: float64

In [7]:
# bin and cut the case_pct column into 2 classifications
q = df['case_pct'].quantile(.9)
bins = [0, q , 40]
labels = ['low','high']
df['case_class'] = pd.cut(df['case_pct'], bins, labels = labels)
df['case_class']

FIPS
1001      low
1009      low
1013      low
1015      low
1017      low
         ... 
48131     low
48505     low
48507    high
48127    high
48247     low
Name: case_class, Length: 3058, dtype: category
Categories (2, object): ['low' < 'high']

In [8]:
df['case_class'].value_counts()

low     2728
high     306
Name: case_class, dtype: int64

In [9]:
#drop unneeded columns
df = df.drop('case_pct', axis =1)
df

,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_PCI,E_NOHSDP,E_AGE65,E_AGE17,...,Neuroticism,Openness,Religiosity,Risk Taking,Selflessness,Tolerance,Work Ethic,dem_pct,first_year_cases,case_class
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,594.443459,55200,23315,21115,8422,1065,29372,4204,8050,13369,...,77.925476,78.222354,91.106719,53.333333,82.142857,70.000000,60.380952,27.018365,6589,low
1009,644.830460,57645,24222,20600,8220,909,22656,7861,10233,13468,...,78.764620,78.193105,92.045455,57.603815,79.307632,64.953288,76.000000,9.569378,6444,low
1013,776.838201,20025,10026,6708,4640,567,20430,2141,3806,4566,...,78.563680,76.109761,76.623924,69.058104,79.956648,67.920284,72.773953,41.789629,2097,low
1015,605.867251,115098,53682,45033,20819,4628,24706,12620,19386,25196,...,79.439032,79.955121,77.918741,54.063568,76.745724,67.456150,68.292794,29.845243,14224,low
1017,596.560643,33826,16981,13516,5531,773,22827,4383,6409,7006,...,76.995358,78.156771,75.891100,67.343775,79.128558,66.397785,69.554441,41.644857,3488,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48131,1793.476183,11355,5592,3511,2751,482,17864,2386,2025,2962,...,79.125428,78.895880,76.629575,60.576045,73.670302,64.571017,68.007770,50.959826,1214,low
48505,998.411980,14369,6388,4405,5609,621,17228,3226,1999,4835,...,79.355639,79.572483,74.378252,77.443239,76.386871,74.001471,73.609838,47.134744,1760,low
48507,1297.406535,12131,4344,3509,4150,421,13350,2719,1665,3583,...,78.392216,76.024682,75.848196,76.967659,77.303576,70.010162,71.121990,65.403060,1844,high


In [10]:
#turn case % classifications into binary 
df = pd.get_dummies(df, columns = ['case_class'])
df

,AREA_SQMI,E_TOTPOP,E_HU,E_HH,E_POV,E_UNEMP,E_PCI,E_NOHSDP,E_AGE65,E_AGE17,...,Openness,Religiosity,Risk Taking,Selflessness,Tolerance,Work Ethic,dem_pct,first_year_cases,case_class_low,case_class_high
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,594.443459,55200,23315,21115,8422,1065,29372,4204,8050,13369,...,78.222354,91.106719,53.333333,82.142857,70.000000,60.380952,27.018365,6589,1,0
1009,644.830460,57645,24222,20600,8220,909,22656,7861,10233,13468,...,78.193105,92.045455,57.603815,79.307632,64.953288,76.000000,9.569378,6444,1,0
1013,776.838201,20025,10026,6708,4640,567,20430,2141,3806,4566,...,76.109761,76.623924,69.058104,79.956648,67.920284,72.773953,41.789629,2097,1,0
1015,605.867251,115098,53682,45033,20819,4628,24706,12620,19386,25196,...,79.955121,77.918741,54.063568,76.745724,67.456150,68.292794,29.845243,14224,1,0
1017,596.560643,33826,16981,13516,5531,773,22827,4383,6409,7006,...,78.156771,75.891100,67.343775,79.128558,66.397785,69.554441,41.644857,3488,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48131,1793.476183,11355,5592,3511,2751,482,17864,2386,2025,2962,...,78.895880,76.629575,60.576045,73.670302,64.571017,68.007770,50.959826,1214,1,0
48505,998.411980,14369,6388,4405,5609,621,17228,3226,1999,4835,...,79.572483,74.378252,77.443239,76.386871,74.001471,73.609838,47.134744,1760,1,0
48507,1297.406535,12131,4344,3509,4150,421,13350,2719,1665,3583,...,76.024682,75.848196,76.967659,77.303576,70.010162,71.121990,65.403060,1844,0,1


In [11]:
X = df.drop(columns= ['case_class_low','case_class_high','first_year_cases']).values
y= df['case_class_high'].values

In [12]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size = .7, random_state=78)

In [13]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
rf_model = RandomForestClassifier(n_estimators=256, random_state=78) 

In [15]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [16]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [17]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [18]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,824,5
Actual 1,88,1


In [19]:
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier()
model = model.fit(X_train_scaled, y_train)

In [20]:
# Making predictions using the testing data.
predictions = model.predict(X_test_scaled)
predictions

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,

In [21]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,751,78
Actual 1,68,21


In [22]:
# We can sort the features by their importance.
X = df.drop(['case_class_low','case_class_high'], axis=1).columns
importance_df = pd.DataFrame(sorted(zip(model.feature_importances_, X), reverse=True))
importance_df

,0,1
0,0.047373,EP_GROUPQ
1,0.039828,Empathy
2,0.038898,Selflessness
3,0.037371,Agreeableness
4,0.036243,Employment Rate
...,...,...
97,0.000000,EP_NOHSDP
98,0.000000,EP_MUNIT
99,0.000000,EPL_UNEMP
100,0.000000,EPL_PCI


In [24]:
impact = importance_df.loc[importance_df[0]>=0]
impact_col = impact[1].to_list()
impact_col

['EP_GROUPQ',
 'Empathy',
 'Selflessness',
 'Agreeableness',
 'Employment Rate',
 'EP_LIMENG',
 'Income Per Capita',
 'Tolerance',
 'RPL_THEME4',
 'EP_SNGPNT',
 'RPL_THEME2',
 'Openness',
 'EPL_CROWD',
 'EPL_NOHSDP',
 'Risk Taking',
 'Neuroticism',
 'Entrepreneurship',
 'Collectivism',
 'EP_MINRTY',
 'E_GROUPQ',
 'EP_MOBILE',
 'SPL_THEMES',
 'EPL_AGE17',
 'EP_AGE17',
 'Extraversion',
 'E_UNINSUR',
 'E_HH',
 'E_POV',
 'EPL_POV',
 'RPL_THEMES',
 'F_GROUPQ',
 'Hopefulness',
 'AREA_SQMI',
 'dem_pct',
 'SPL_THEME1',
 'Belief In Science',
 'EPL_SNGPNT',
 'F_TOTAL',
 'EPL_MUNIT',
 'EPL_MOBILE',
 'E_AGE65',
 'E_HU',
 'EP_UNEMP',
 'EPL_MINRTY',
 'EPL_NOVEH',
 'E_SNGPNT',
 'E_DISABL',
 'E_CROWD',
 'Conscientiousness',
 'EPL_GROUPQ',
 'SPL_THEME2',
 'RPL_THEME3',
 'F_THEME4',
 'EPL_LIMENG',
 'EPL_DISABL',
 'Gender Equality',
 'EP_NOVEH',
 'EPL_AGE65',
 'SPL_THEME4',
 'E_LIMENG',
 'EP_AGE65',
 'Religiosity',
 'F_AGE65',
 'RPL_THEME1',
 'EP_CROWD',
 'EP_DISABL',
 'EP_UNINSUR',
 'Work Ethic',
 'SPL_

In [25]:
impact_df = df[impact_col]
impact_df

,EP_GROUPQ,Empathy,Selflessness,Agreeableness,Employment Rate,EP_LIMENG,Income Per Capita,Tolerance,RPL_THEME4,EP_SNGPNT,...,E_MINRTY,E_DAYPOP,E_AGE17,EP_POV,EP_PCI,EP_NOHSDP,EP_MUNIT,EPL_UNEMP,EPL_PCI,Conflict Awareness
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,1.0,83.308726,82.142857,86.279655,94.7,0.8,26168.0,70.000000,0.3741,7.5,...,13788,37301,13369,15.4,29372.0,11.3,3.8,0.2745,0.2860,63.444323
1009,0.9,81.521844,79.307632,85.548096,94.5,1.7,21033.0,64.953288,0.1741,7.0,...,7413,40036,13468,14.4,22656.0,19.8,0.9,0.2611,0.7561,63.136502
1013,1.6,81.846372,79.956648,88.377418,92.9,0.5,19011.0,67.920284,0.6259,10.5,...,9641,17280,4566,23.5,20430.0,15.4,1.3,0.6968,0.8879,60.788532
1015,2.7,82.318215,76.745724,85.039324,93.3,1.0,22231.0,67.456150,0.6492,10.4,...,31675,117894,25196,18.6,24706.0,15.9,3.7,0.8850,0.6076,47.565763
1017,1.5,80.922581,79.128558,86.224666,94.5,0.1,21532.0,66.397785,0.7943,9.7,...,14954,27176,7006,16.6,22827.0,18.6,4.0,0.4166,0.7465,65.485423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48131,4.4,59.975800,73.670302,87.364932,88.9,5.7,19853.0,64.571017,0.8978,15.8,...,10327,9592,2962,25.6,17864.0,32.9,0.5,0.9516,0.9608,43.491311
48505,0.2,77.457883,76.386871,87.316650,89.0,24.4,16007.0,74.001471,0.6966,17.1,...,13754,10924,4835,39.5,17228.0,40.0,0.6,0.9576,0.9736,46.302376
48507,1.6,79.709157,77.303576,85.386374,85.9,8.0,13393.0,70.010162,0.9331,16.1,...,11529,10672,3583,34.8,13350.0,38.0,1.7,0.8611,0.9978,60.553983


In [26]:
X = impact_df.values
y= df['case_class_high'].values

In [27]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size = .7, random_state=78)
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [28]:
from sklearn.ensemble import GradientBoostingClassifier
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
    learning_rate=learning_rate,
    max_features=5,
    max_depth=3,
    random_state=0)
    classifier.fit(X_train_scaled, y_train)
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(
           classifier.score(
           X_train_scaled,
           y_train)))
    print("Accuracy score (validation): {0:.3f}".format(
           classifier.score(
           X_test_scaled,
           y_test)))

Learning rate:  0.05
Accuracy score (training): 0.899
Accuracy score (validation): 0.903
Learning rate:  0.1
Accuracy score (training): 0.907
Accuracy score (validation): 0.902
Learning rate:  0.25
Accuracy score (training): 0.920
Accuracy score (validation): 0.904
Learning rate:  0.5
Accuracy score (training): 0.936
Accuracy score (validation): 0.894
Learning rate:  0.75
Accuracy score (training): 0.947
Accuracy score (validation): 0.878
Learning rate:  1
Accuracy score (training): 0.943
Accuracy score (validation): 0.874


In [29]:
classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=0.25, max_features=5, max_depth=3, random_state=0)

classifier.fit(X_train_scaled, y_train)
predictions = classifier.predict(X_test_scaled)

In [30]:
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.9041394335511983


In [31]:
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
   cm, index=["Actual 0", "Actual 1"],
   columns=["Predicted 0", "Predicted 1"]
)
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,827,2
Actual 1,86,3


In [32]:
print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

           0       0.91      1.00      0.95       829
           1       0.60      0.03      0.06        89

    accuracy                           0.90       918
   macro avg       0.75      0.52      0.51       918
weighted avg       0.88      0.90      0.86       918

